### [10\. Regular Expression Matching](https://leetcode.com/problems/regular-expression-matching/)

Difficulty: **Hard**


Given an input string (`s`) and a pattern (`p`), implement regular expression matching with support for `'.'` and `'*'`.

```
'.' Matches any single character.
    單一字元
'*' Matches zero or more of the preceding element.
    將前面的字元重複任意次(0也可以)
```

The matching should cover the **entire** input string (not partial).

**Note:**

*   `s` could be empty and contains only lowercase letters `a-z`.
*   `p` could be empty and contains only lowercase letters `a-z`, and characters like `.` or `*`.

**Example 1:**

```
Input:
s = "aa"
p = "a"
Output: false
Explanation: "a" does not match the entire string "aa".
```

**Example 2:**

```
Input:
s = "aa"
p = "a*"
Output: true
Explanation: '*' means zero or more of the preceding element, 'a'. Therefore, by repeating 'a' once, it becomes "aa".
```

**Example 3:**

```
Input:
s = "ab"
p = ".*"
Output: true
Explanation: ".*" means "zero or more (*) of any character (.)".
```

**Example 4:**

```
Input:
s = "aab"
p = "c*a*b"
Output: true
Explanation: c can be repeated 0 times, a can be repeated 1 time. Therefore, it matches "aab".
```

**Example 5:**

```
Input:
s = "mississippi"
p = "mis*is*p*."
Output: false
```

# Solution 1: Queue 失敗

遍歷 Pattern 總是要看下一個 (如果有的話) 是不是 \*

1. 如果下一個不是 \*

* [a-z]: 與 s 的字首比較, 正確就各自消除
* . : 各自消除
* 不符合就回傳 False

2. 如果下一個是 \*

* 消除 s 前面, 所有與 p 字首一樣的字, 沒有也沒關係

當 p 已經空掉, s 卻還有東西, 回傳 False

## 問題

1. 使用 while 時, 盡量以 break 跳出迴圈後再判斷 True / False

2. \* 的可變性, 牽涉到太多可能, 無法以迴圈解決
```
    s: aaa
    p: a*a
    return: True
```

In [0]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        while p and s:
            cur = p[0]

            if len(p) > 1:
                nxt = p[1]
                if nxt != '*':
                    if cur == '.' or cur == s[0]:
                        p = p[1:]
                        s = s[1:]
                    else:
                        break
                else:
                    if cur == '.':
                        s = ''
                        p = p[2:]
                        break
                    
                    n = len(s)
                    cout = 0
                    while cout < n and s[cout] == cur:
                        cout += 1
                    s = s[cout:]
                    p = p[2:]

            if s == '' or p == '':
                break

            if cur == '.' or cur == s[0]:
                p = p[1:]
                s = s[1:]

            print(f's:{s}, p:{p}')
            
        print(f's:{s}, p:{p}')
        if p or s:
            return False
        else:
            return True

## Solution 2: 2D-DP

61.14%

[參考貼文](https://leetcode.com/problems/regular-expression-matching/discuss/5723/My-DP-approach-in-Python-with-comments-and-unittest)

[參考影片](https://youtu.be/l3hda49XcDE)

2D-DP
```
dp[len(s)][len(p)]

dp[i][j]: 在 s 中取到第 i 個字, 在 p 中取到第 j 個字, 互相比較

對應的字元會變成 p[i-1] s[j-1]
```

Initial:
```
dp[0][0]: True
Pattern 跟空字串比對: * 可能可以是對應到空字串
```

Transform

當前 Pattern 不是 \*
```
上次比對(左上)要是 True, 而且現在的比對也要是 True
```

當前 Pattern 是 \*
```
1. 根本沒有 p[i-2] 的字元: dp[i-2][j]

2. 重複一次: dp[i-1][j]

3. 重複多次: dp[i][j-1]

至少重複一次: dp[i-1][j] or (s[j - 1] == p[i - 2] || p[i - 2] == '.') and dp[i][j - 1]
```

結果
```
return dp[-1][-1]
```

In [0]:
class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        
        dp = [[False for _ in range(len(s) + 1)] for _ in range(len(p) + 1)]
        
        dp[0][0] = True
        
        # 尋找可能為空的 * 
        for i in range(2, len(p) + 1):
             dp[i][0] = dp[i-2][0] and p[i-1] == '*'
        
        
        for i in range(1, len(p) + 1):
            for j in range(1, len(s) + 1):
                
                if p[i-1] != '*':
                    dp[i][j] = dp[i-1][j-1] and \
                    (p[i-1] == s[j-1] or p[i-1] == '.')
            
                else:
                    dp[i][j] = dp[i-2][j] or dp[i-1][j] # 沒有重複 or 重複一次
                    
                    if p[i-2] == s[j-1] or p[i-2] == '.':
                        dp[i][j] = dp[i][j] | dp[i][j-1] # 重複多次
            
        return dp[-1][-1]

s:, p:a


False

## Solution 3: Recursive + Cache

94.52%

倒著遍歷

將 dictionary 當作 Cache 來使用

In [0]:
class Solution:
    cache = {}
    def isMatch(self, s: str, p: str) -> bool:
        # 出現過的模式
        if (s, p) in self.cache:
            return self.cache[(s, p)]
        # pattern 已經為空
        if not p:
            return not s
        if p[-1] == '*':
            # 沒有重複
            if self.isMatch(s,p[:-2]):
                self.cache[(s, p)] = True
                return True
            # 有重複
            if s and (s[-1] == p[-2] or p[-2] == '.') and self.isMatch(s[:-1], p):
                self.cache[(s, p)] = True
                return True
        if s and (p[-1] == s[-1] or p[-1] == '.') and self.isMatch(s[:-1], p[:-1]):
            self.cache[(s, p)] = True
            return True
        self.cache[(s, p)] = False
        return False